#### Langchain, openai key and HUggingFace API key

In [27]:
import warnings
warnings.filterwarnings("ignore")
from langchain_openai import ChatOpenAI
from constant import openai_key
from huggingface_Api import HUGGINGFACEHUB_API_TOKEN

#### Call the openai env key globally

In [28]:
import os
os.environ["OPENAI_API_KEY"] = openai_key

#### From Langchain Initialize the OpenAI LLM

In [29]:
llm = ChatOpenAI(temperature=0.8) # More the value towards 1 more random the LLM would be

##### Testing

In [30]:
text = " what is the capital of india"
print(llm.predict(text)) # also we can use (llm.invoke)

The capital of India is New Delhi.


#### From Huggingface 
##### We can use opensource model from Huggingface
##### Declare the env globally

In [31]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

#### We use "google/flan-t5-large" it is a opensource model from Huggingface, also we are testing it below

In [32]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-large")#,model_kwargs={"temperature":0,"max_lenght":64})

In [34]:
text_hug="what is the weather in new delhi and kolkata"
print(llm_huggingface.predict(text_hug))

The average temperature in New Delhi is 1°C, while in Kolkata it is 


#### compare it with OpenAI LLM

In [9]:
text_openai_llm="what is the weather in new delhi and kolkata"
print(llm.predict(text_openai_llm))

I am sorry, I do not have real-time weather information. You can check the weather in New Delhi and Kolkata by using a reliable weather website or app.


#### Prompt Template

In [35]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=["country"],
               template="Tell me something about the country {country}")

prompt_template.format(country="India")

'Tell me something about the country India'

In [36]:
## To run the prompt and the llm along with it we need llm chain.

from langchain.chains import LLMChain

chain = LLMChain (llm=llm, prompt=prompt_template) ## Open AI LLM

print(chain.run("India"))

India is the seventh-largest country in the world by land area and the second-most populous country, with over 1.3 billion people. It is located in South Asia and is known for its rich cultural heritage, diverse languages, and vibrant traditions. India is a federal parliamentary democratic republic with a President as its head of state and a Prime Minister as its head of government. The country is also known for its Bollywood film industry, spicy cuisine, and ancient history dating back thousands of years. India is a major player in the global economy and is known for its IT industry, agriculture, and textile manufacturing.


#### Combine Multiple Chain using simple Sequential chain

In [37]:
### Creating 2 chains
capital_prompt = PromptTemplate(input_variables = ["country"],
                              template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm = llm, prompt = capital_prompt)

famous_prompt = PromptTemplate(input_variables = ["capital"],
                               template="Please tell me the famous places in {capital}")

famous_chain  = LLMChain(llm = llm, prompt = famous_prompt)

In [38]:
from langchain.chains import SimpleSequentialChain
simple_seq_chain = SimpleSequentialChain(chains=[capital_chain,famous_chain])
print(simple_seq_chain.run("India"))

Some famous places in New Delhi, the capital of India, include:

1. Red Fort
2. India Gate
3. Qutub Minar
4. Lotus Temple
5. Humayun's Tomb
6. Jama Masjid
7. Akshardham Temple
8. Rashtrapati Bhavan (President's House)
9. Chandni Chowk
10. National Museum

These are just a few of the many iconic landmarks and destinations in New Delhi that attract tourists from around the world.


### Sequential Chain

In [39]:
### Creating 2 chains
capital_prompt = PromptTemplate(input_variables = ["country"],
                              template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm = llm, prompt = capital_prompt, output_key = "capital")

famous_prompt = PromptTemplate(input_variables = ["capital"],
                               template="Please tell me the famous places in {capital}")

famous_chain  = LLMChain(llm = llm, prompt = famous_prompt, output_key = "places")

In [40]:
from langchain.chains import SequentialChain

seq_chain = SequentialChain(chains = [capital_chain, famous_chain],
                            input_variables=["country"], output_variables= ["capital", "places"])

print(chain({"country":"India"}))

{'country': 'India', 'text': 'India is the seventh-largest country in the world by land area and the second-most populous country, with over 1.3 billion people. It is located in South Asia and is known for its rich history, diverse culture, and vibrant traditions. India is also home to a wide variety of languages, religions, and cuisines, making it a truly unique and dynamic country. Additionally, India is known for its contributions to the fields of science, technology, and art, and has a rapidly growing economy that has made it one of the fastest-growing major economies in the world.'}


#### Chatmodels (LangChain) with ChatOpenAI

In [41]:
from langchain.chat_models import ChatOpenAI
# so when we use chatmodels we use 3 schemas
from langchain.schema import HumanMessage, SystemMessage, AIMessage
# Reinitialing ChatLLM model from OpenAI
chat_llm = ChatOpenAI(temperature=0.6,model="gpt-3.5-turbo")

In [42]:
### Let use the 3 schemas HumanMessage(Human Input), SystemMessage(Instruction to the chatbot),AIMessage (Output of the chatbot)  
chat_llm([
    SystemMessage(content="You are a Comedian AI assitant"), # this is what I tell my chatbot to behave like
    HumanMessage(content="Let me some jokes of AI") # Input from Human
])

AIMessage(content='Sure, here are some AI jokes for you:\n\n1. Why did the robot go to school? Because it wanted a little byte of knowledge!\n2. How does a computer say goodbye? "Byte, byte, byte!"\n3. Why was the computer cold? It left its Windows open!\n4. Why did the robot break up with his girlfriend? She couldn\'t handle his motherboard issues!\n5. How do robots send messages? By Bluetooth!\n\nI hope these jokes bring a smile to your face!')

### Prompt template + LLM + Output Parse

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser ## I want to modify any output of an LLM before hand we use Output parser

### for output parser

In [44]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")

#### For Chat prompt

In [45]:
template = "You are a helpfull assistant. Whenever a user gives input, you should generate 5 words synonyms in comma seperated"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [46]:
## To run this chatprompt we need chain 
## Here chatprompt is the prompt along with system and human message
## chatllm is the chat llm from openai
## commaseperatedoutput is the class from the Output parser
output_parser_chain = chatprompt | chat_llm | Commaseperatedoutput()

In [47]:
output_parser_chain.invoke({"text": "Interligent"})

['Intelligent', ' clever', ' smart', ' brainy', ' astute']